# Batch Lamda1050 jump correction script

### Author: Vinicio Soto, CICIMA
### This script works with raw Lambdas1050's ASC files in batch. It corrects Lambdas1050's detector jump between 857 nm and 858 nm 

### Dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os

### Instructions:

Write your archive folder's path 

In [2]:
#Lists all archives in folder
transflectance_file_folder_data_path = r"C:\Users\EstebanSoto\Documents\Estudio Optico Escarabajos\new_data\2023NOV_TRANSFLECTANCE\transflectance"
transflectance_file_list = os.listdir(transflectance_file_folder_data_path)
transmittance_file_folder_data_path = r"C:\Users\EstebanSoto\Documents\Estudio Optico Escarabajos\new_data\2023NOV_TRANSMITTANCE\transmittance"
transmittance_file_list = os.listdir(transmittance_file_folder_data_path)

#file extension
file_extension = ".ASC"

In [3]:
# filters a list of strings to create a new list containing only the elements that end with ".ASC"

def filter_substring_elements(path_strings, substring):
    filtered_paths = [path for path in path_strings if substring in path]
    return filtered_paths

# Filtering elements ending with ".ASC"
transflectance_result_list = filter_substring_elements(transflectance_file_list, file_extension)
transmittance_result_list = filter_substring_elements(transmittance_file_list, file_extension)
# Displaying the filtered list
print(transflectance_result_list)
print(transmittance_result_list)

['TXRX1-C.AURI.INBI.ASC', 'TXRX1-C.BOUC.CO23.ASC', 'TXRX1-C.BOUC.INB5423.ASC', 'TXRX1-C.BOUC.INB5610.ASC', 'TXRX1-C.CHRY.6713.ASC', 'TXRX1-C.CHRY.MV73.ASC', 'TXRX1-C.CHRY.MV84.ASC', 'TXRX1-C.CUPR.MV23.ASC', 'TXRX1-C.KALI.LA22.ASC', 'TXRX1-C.OPTI.8.17.ASC', 'TXRX1-C.RESP.MV23.ASC', 'TXRX1-M.HIRT.DIUR .ASC', 'TXRX1-MACRASPIS.SP.ASC', 'TXRX1-NI#0001.ASC', 'TXRX1-NI#0002.ASC', 'TXRX1-P.GRAT.3610 .ASC', 'TXRX1-PELID.001 .ASC', 'TXRX1-PELID.002.ASC', 'TXRX1-PLATY.001.ASC', 'TXRX1-S.BELT.1063.ASC', 'TXRX1-S.GLAB.0620.ASC', 'TXRX1-S.SIGN.PREG.ASC', 'TXRX2-C.AURI.INBI.ASC', 'TXRX2-C.BOUC.CO23.ASC', 'TXRX2-C.BOUC.INB54230.ASC', 'TXRX2-C.BOUC.INB5610.ASC', 'TXRX2-C.CHRY.6713.ASC', 'TXRX2-C.CHRY.MV73.ASC', 'TXRX2-C.CHRY.MV84.ASC', 'TXRX2-C.CUPR.MV23.ASC', 'TXRX2-C.KALI.LA22.ASC', 'TXRX2-C.OPTI.8.17.ASC', 'TXRX2-C.RESP.MV23.ASC', 'TXRX2-M.HIRT.DIUR.ASC', 'TXRX2-MACRASPIS.SP.ASC', 'TXRX2-NI#0001.ASC', 'TXRX2-NI#0002.ASC', 'TXRX2-P.GRAT.3610.ASC', 'TXRX2-PELID.001 .ASC', 'TXRX2-PELID.002.ASC', 'TXRX2

### Correction process function

In [4]:
def jump_correction(file_folder_data_path, filename, measurement_type):
    #Load archive
     #constants
    markersize = 1 #Marker size for plots

    # read text file into pandas DataFrame 
    full_path= file_folder_data_path + "\\" + filename
    df = pd.read_csv(full_path, sep="	",header=90,  
                     names=["wavelength", measurement_type]) 
    #Data cleaning
    #Replace commas with dots
    df = df.replace(',','.', regex=True)
    #print("1")
    #print(df.dtypes)

    #convert columns to float
    df['wavelength'] = df['wavelength'].astype(float)
    df[measurement_type] = df[measurement_type].astype(float)
    

    #This code finds the rows before and after the detector change
    #freq1 is the frequency before the change and freq2 is the frequency after the change
    wavelenght1 = 857.000000 #857
    wavelenght2 = 858.000000 #858
    #print("Loc")
    row1 = df.loc[ df['wavelength'] == wavelenght1]
    row2 = df.loc[ df['wavelength'] == wavelenght2]
    row22 = df.loc[ df['wavelength'] == wavelenght2 +1]
    row23 = df.loc[ df['wavelength'] == wavelenght2+2]
    row24 = df.loc[ df['wavelength'] == wavelenght2+3]
    row25 = df.loc[ df['wavelength'] == wavelenght2+4]
    
    #This code finds the values corresponding to those frequencies and creates a correction factor
    #print("dataframe new")
    #print(row1)
    data1 = row1.iat[0,1]
    data2 = row2.iat[0,1]
    data22 = row22.iat[0,1]
    data23 = row23.iat[0,1]
    data24 = row23.iat[0,1]
    data25 = row23.iat[0,1]
    
    data2_avg = (data2 + data22+data23+data24+data25)/5
    #print(data1)
    #print(data2)
    correction_factor= data2_avg/data1 *0.98
    #print(correction_factor)
    #Multiply all frequencies equal or greater than freq2 by correction_factor
    df2 = df
    df2.loc[df2["wavelength"] <= wavelenght2, [measurement_type]] *= correction_factor
    #Plot modified spectra
    #df2.plot(x='wavelength', y=measurement_type, kind='scatter', s=markersize, title = "ADJUSTED "+filename,grid=True)	
    #Save to a file delimited by tabs
    my_numpy = df.to_numpy()
    #Create new folder
    corrected_path = file_folder_data_path + "\\jump_corrected_files\\"
    if not os.path.exists(corrected_path):
        os.mkdir(corrected_path)
    #saves file
    new_archive_name = corrected_path +filename.replace('.ASC', ''+'.txt')
    np.savetxt(new_archive_name, my_numpy,fmt='%s', delimiter="\t")

### Apply code to all files ending in .ASC in folder 

In [5]:
measurement_type = "transflectance" 

for file in transflectance_result_list:
    jump_correction(file_folder_data_path, file,measurement_type)
    
measurement_type = "transmittance" 

for file in transmittance_result_list:
    jump_correction(transmittance_file_folder_data_path, file, measurement_type)